<a href="https://colab.research.google.com/github/Vinicius667/ML/blob/main/3%20CIFAR_10/cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys

In [2]:
if  os.getenv("COLAB_RELEASE_TAG"):
  is_running_on_drive = True
else:
  is_running_on_drive = False

In [3]:
if is_running_on_drive:
  packages_to_install = ['torchshape','torchstat']

  for package in packages_to_install:
    os.system(f'pip install {package}')
    from google.colab import drive, files
    dir = "/content/gdrive/MyDrive/Colab_Notebooks/ML/CIFAR_10"
    drive.mount('/content/gdrive')
    if os.getcwd() != dir:
      os.chdir(dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import torch
import torch.nn.functional as F

#####################
cuda_available = torch.cuda.is_available()

if cuda_available:
    device = 'cuda'
    print(f'device name: {torch.cuda.get_device_name(0)}')
else:
    device = 'cpu'
    print(device)

#####################

import torch.nn as nn
import torch.nn.functional as functional
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import sys
from datetime import  datetime
import pandas as pd
from sklearn.metrics import confusion_matrix
from torchshape import tensorshape
from torchstat import stat


device name: Tesla T4


In [5]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [6]:
class MyCNN:
    def __init__(self,shape) -> None:
        self.shape = tuple(shape)
        self.layers = []
        self.isflatten = False
        print(self.shape)


    def add_conv(self,
        #in_channels: int,
        out_channels: int,
        kernel_size:int, **kwargs):
        layer = nn.Conv2d(self.shape[1],out_channels, kernel_size,**kwargs)
        self.shape = tensorshape(layer,self.shape)
        self.layers.append(layer)
        print(self.shape)

    def add_max_pool(self,kernel_size, stride=None, padding=0, dilation=1,**kwargs):
        layer = nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1,**kwargs)
        self.shape = tensorshape(layer,self.shape)
        self.layers.append(layer)
        print(self.shape)


    def make_flatten(self):
        if not self.isflatten:
            layer = nn.Flatten()
            self.shape = tensorshape(layer,self.shape)
            self.layers.append(layer)
            self.isflatten = True
            print(self.shape)


    def add_fc(self, out_features: int,**kwargs):
        self.make_flatten()
        layer = nn.Linear(self.shape[1],out_features)
        self.shape = tensorshape(layer,self.shape)
        self.layers.append(layer)
        print(self.shape)

    def add_activation(self,function):
        self.layers.append(function)

    def get_model(self):
        return nn.Sequential(*self.layers)

In [7]:
def create_CNN(initial_shape,model_params):
  my_cnn = MyCNN(initial_shape)
  for function, kwargs in model_params:
    if function == "conv":
      my_cnn.add_conv(**kwargs)
    elif function == "activation":
      my_cnn.add_activation(**kwargs)
    elif function == "max_pool":
      my_cnn.add_max_pool(**kwargs)
    elif function == "fc":
      my_cnn.add_fc(**kwargs)
    else:
      raise
  return my_cnn.get_model()


In [8]:
# Composes several transforms together
# Parameters:
    #transforms (list of Transform objects) – list of transforms to compose.
transform = transforms.Compose(
    [
    #Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] if the PIL Image belongs to one of the modes (L, LA, P, I, F, RGB, YCbCr, RGBA, CMYK, 1) or if the numpy.ndarray has dtype = np.uint8
    #In the other cases, tensors are returned without scaling.
    transforms.ToTensor(),

    # For n channels, this transform will normalize each channel of the input torch.*Tensor i.e.
    # output[channel] = (input[channel] - mean[channel]) / std[channel]
    # Parameters:
        # mean (sequence): (mean[1],...,mean[n]),
        # std (sequence): (std[1],..,std[n])
        # inplace
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),

    transforms.RandomHorizontalFlip(),

    ]
)

In [9]:
batch_size = 32

In [10]:
train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True, download=True,transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False, download=True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
num_train_samples = len(train_dataset.targets)
num_test_samples = len(test_dataset.targets)

In [12]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,pin_memory=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True,pin_memory=True,num_workers=2)

In [13]:
classes_names = train_dataset.classes
" - ".join(classes_names)

'airplane - automobile - bird - cat - deer - dog - frog - horse - ship - truck'

In [14]:
aux = pd.Series(train_dataset.targets)
classes_df = aux.value_counts().to_frame().sort_index()
classes_df["Name"] = classes_df.index.map({i:j for i, j in enumerate(classes_names)})
display(classes_df)
aux = pd.Series(test_dataset.targets)
classes_df = aux.value_counts().to_frame().sort_index()
classes_df["Name"] = classes_df.index.map({i:j for i, j in enumerate(classes_names)})

,0,Name
0,5000,airplane
1,5000,automobile
2,5000,bird
3,5000,cat
4,5000,deer
5,5000,dog
6,5000,frog
7,5000,horse
8,5000,ship
9,5000,truck


In [23]:
models = [
  [
  #seed = 42
  # loss: 1.621
  #Accuracy of the network: 77.04 | airplane: 83.10% | automobile: 87.60% | bird: 64.10% | cat: 58.40% | deer: 76.50% | dog: 65.10% | frog: 84.30% | horse: 80.40% | ship: 85.90% | truck: 85.00%
  ["conv",dict(out_channels=128,kernel_size= 3)],
  ['activation',dict(function= nn.ReLU())],
  ['max_pool',dict(kernel_size=2)],
  ["conv",dict(out_channels=256,kernel_size= 3)],
  ['activation',dict(function= nn.ReLU())],
  ['max_pool',dict(kernel_size=3)],
  ['fc',dict(out_features=120)],
  ['activation',dict(function= nn.ReLU())],
  ['fc',dict(out_features=60)],
  ['activation',dict(function= nn.ReLU())],
  ['fc',dict(out_features=10)],
  ['activation',dict(function= nn.Softmax(1))],
  ],

  [
  # seed = 42
  # [1,   200] loss: 1.686
  # Accuracy of the network: 68.74 | airplane: 78.40% | automobile: 83.70% | bird: 0.00% | cat: 61.10% | deer: 74.10% | dog: 65.70% | frog: 85.40% | horse: 74.40% | ship: 82.50% | truck: 82.10%

  # seed = 16
  # loss: 1.682
  # Accuracy of the network: 70.12 | airplane: 73.80% | automobile: 81.80% | bird: 56.90% | cat: 53.70% | deer: 66.20% | dog: 61.20% | frog: 79.10% | horse: 72.00% | ship: 78.60% | truck: 77.90%

  ["conv",dict(out_channels=64,kernel_size= 3)],
  ['activation',dict(function= nn.ReLU())],
  ['max_pool',dict(kernel_size=2)],
  ["conv",dict(out_channels=128,kernel_size= 3)],
  ['activation',dict(function= nn.ReLU())],
  ['max_pool',dict(kernel_size=2)],
  ["conv",dict(out_channels=256,kernel_size= 3)],
  ['activation',dict(function= nn.ReLU())],
  ['max_pool',dict(kernel_size=3)],
  ['fc',dict(out_features=120)],
  ['activation',dict(function= nn.ReLU())],
  ['fc',dict(out_features=60)],
  ['activation',dict(function= nn.ReLU())],
  ['fc',dict(out_features=10)],
  ['activation',dict(function= nn.Softmax(1))],
  ],


  [

  ["conv",dict(out_channels=32,kernel_size= 3)],
  ['activation',dict(function= nn.ReLU())],
  ["conv",dict(out_channels=32,kernel_size= 3)],
  ['activation',dict(function= nn.ReLU())],
  ['max_pool',dict(kernel_size=2)],

  ["conv",dict(out_channels=64,kernel_size= 3)],
  ['activation',dict(function= nn.ReLU())],
  ["conv",dict(out_channels=64,kernel_size= 3)],
  ['activation',dict(function= nn.ReLU())],

  ['fc',dict(out_features=240)],
  ['activation',dict(function= nn.ReLU())],
  ['fc',dict(out_features=120)],
  ['activation',dict(function= nn.ReLU())],
  ['fc',dict(out_features=60)],
  ['activation',dict(function= nn.ReLU())],
  ['fc',dict(out_features=10)],
  ['activation',dict(function= nn.Softmax(1))],
  ],

]

In [24]:
criterion = nn.CrossEntropyLoss()
epochs = 30
momentum = 0.5
learning_rate = 1e-1
seed = 42

for num,model_params in enumerate(models):
  if num<2:
    continue
  model_state_dict_path = f'./model_{num}_seed_{seed}_state_dict_path'
  print("=*"*20)
  print(f"model num = {num}")
  torch.manual_seed(seed)
  model = create_CNN(initial_shape=(16, 3, 32, 32),model_params=model_params).to(device)
  if os.path.exists(model_state_dict_path):
    print(f"Carregando state dict: {model_state_dict_path}")
    model.load_state_dict(torch.load(model_state_dict_path))
  else:
    #torch.save(model.state_dict(), model_state_dict_path)
    print(f"Criando state dict: {model_state_dict_path}")

  optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate,momentum=momentum)
  model.train(True)
  for epoch in range(epochs):
    running_loss = 0.0
    learning_rate = learning_rate*0.5
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate,momentum=momentum)
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = F.one_hot(labels,10).to(torch.float32).to(device)
      optimizer.zero_grad()
      output = model(images)
      loss = criterion(output,labels)
      running_loss +=loss.item()
      loss.backward()
      optimizer.step()
      if(i%200) == 199:
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
        running_loss = 0.0

        model.eval()
        with torch.inference_mode():
          n_corret = 0
          n_samples = 0
          n_class_correct = [0 for i in range(10)]
          n_class_samples = [0 for i in range(10)]

          for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images).to(device)
            predicted = torch.argmax(output,1)
            n_samples+= labels.size(0)
            n_corret += (predicted == labels).sum().item()

            for i in range(labels.shape[0]):
              label = labels[i]
              pred = predicted[i]
              if (label == pred):
                n_class_correct[label] += 1
              n_class_samples[label] += 1
          acc = 100 * n_corret/n_samples
          print(f"Accuracy of the network: {acc:.2f} | ",end="")

          #idxmin = pd.Series(n_class_correct).idxmin()
          for idx in range(10):
            acc = 100 * n_class_correct[idx]/n_class_samples[idx]
            print(f'{classes_names[idx]}: {acc:.2f}% | ',end="")
          print("")


torch.save(model.state_dict(), model_state_dict_path)


=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
model num = 2
(16, 3, 32, 32)
(16, 16, 30, 30)
(16, 32, 28, 28)
(16, 32, 14, 14)
(16, 64, 12, 12)
(16, 64, 6, 6)
(16, 128, 4, 4)
(16, 2048)
(16, 240)
(16, 120)
(16, 60)
(16, 10)
Criando state dict: ./model_2_seed_16_state_dict_path
[1,   200] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[1,   400] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[1,   600] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[1,   800] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe335f8ea70>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe335f8ea70>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/

[22,  1400] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[23,   200] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[23,   400] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[23,   600] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[23,   800] loss: 2.302
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.0

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe335f8ea70>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe335f8ea70>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/

[23,  1200] loss: 2.302
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[23,  1400] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[24,   200] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[24,   400] loss: 2.302
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[24,   600] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.0

...

Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[28,  1200] loss: 2.303


Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe335f8ea70>
Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fe335f8ea70>  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__

    Traceback (most recent call last):
self._shutdown_workers()  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    
self._shutdown_workers()  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers

      File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
      File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
assert self._parent_pid == os.getpid(), 'can only test a

Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[28,  1400] loss: 2.303


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe335f8ea70>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe335f8ea70>self._shutdown_workers()

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():AssertionError: 
assert self._parent_pid == os.getpid(), 'can only test a child process'can only test a child process
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fe335f8ea70>
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
      File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", lin

Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[29,   200] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[29,   400] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[29,   600] loss: 2.302
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship: 0.00% | truck: 0.00% | 
[29,   800] loss: 2.303
Accuracy of the network: 10.00 | airplane: 100.00% | automobile: 0.00% | bird: 0.00% | cat: 0.00% | deer: 0.00% | dog: 0.00% | frog: 0.00% | horse: 0.00% | ship

In [ ]:
torch.save(model.state_dict(), model_state_dict_path)

In [ ]:
num_train_iter = epochs*num_train_samples/batch_size
num_train_iter

In [ ]:
output

In [ ]:
cf_matrix = confusion_matrix(labels, predicted)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])

In [ ]:
torch.save(model.state_dict(), datetime.today().strftime("./cifar_10_model_%Y_%m_%d_%H_%M_%S"))
